# London Bike Rides

### Introduction

This project uses the **London Bike Sharing dataset**, which combines hourly Santander Cycles usage in London (powered by TfL Open Data) with weather conditions and holiday/weekend information from 2015–2016. The goal is to model and predict future bike-share demand based on factors like temperature, humidity, wind, season, holidays, and day type.

In [2]:
#! pip install pandas
#! pip install kaggle
#! pip install openpyxl

In [3]:
# Importing the libraries that are used in this project
import pandas as pd

# Use this to extract the file downloaded from Kaggle
import zipfile

# Use this to download the dataset programmatically from Kaggle
import kaggle


### Collecting Data Programmatically

In [4]:
# Download dataset from kaggle using the Kaggle API
!kaggle datasets download -d hmavrodiev/london-bike-sharing-dataset

Dataset URL: https://www.kaggle.com/datasets/hmavrodiev/london-bike-sharing-dataset
License(s): other
london-bike-sharing-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
# Extract the file from the downloaded zip file
zipfile_name = 'london-bike-sharing-dataset.zip'
with zipfile.ZipFile(zipfile_name, 'r') as file:
    file.extractall()

In [6]:
# Read in the CSV file as a pandas dataframe 
bike_rides = pd.read_csv("london_merged.csv")

### Assessing and Cleaning the dataset

In [7]:
bike_rides.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17414 entries, 0 to 17413
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   timestamp     17414 non-null  object 
 1   cnt           17414 non-null  int64  
 2   t1            17414 non-null  float64
 3   t2            17414 non-null  float64
 4   hum           17414 non-null  float64
 5   wind_speed    17414 non-null  float64
 6   weather_code  17414 non-null  float64
 7   is_holiday    17414 non-null  float64
 8   is_weekend    17414 non-null  float64
 9   season        17414 non-null  float64
dtypes: float64(8), int64(1), object(1)
memory usage: 1.3+ MB


In [8]:
bike_rides.shape

(17414, 10)

In [9]:
bike_rides.head(10)

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0,0.0,1.0,3.0
1,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0,0.0,1.0,3.0
2,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0,0.0,1.0,3.0
3,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0,0.0,1.0,3.0
4,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0,0.0,1.0,3.0
5,2015-01-04 05:00:00,46,2.0,2.0,93.0,4.0,1.0,0.0,1.0,3.0
6,2015-01-04 06:00:00,51,1.0,-1.0,100.0,7.0,4.0,0.0,1.0,3.0
7,2015-01-04 07:00:00,75,1.0,-1.0,100.0,7.0,4.0,0.0,1.0,3.0
8,2015-01-04 08:00:00,131,1.5,-1.0,96.5,8.0,4.0,0.0,1.0,3.0
9,2015-01-04 09:00:00,301,2.0,-0.5,100.0,9.0,3.0,0.0,1.0,3.0


In [10]:
# Count the unique values in the weather_code column
bike_rides.weather_code.value_counts()

weather_code
1.0     6150
2.0     4034
3.0     3551
7.0     2141
4.0     1464
26.0      60
10.0      14
Name: count, dtype: int64

In [11]:
# Count the unique values in the season column
bike_rides.season.value_counts()

season
0.0    4394
1.0    4387
3.0    4330
2.0    4303
Name: count, dtype: int64

In [12]:
# Specifying the column names for better understanding
new_columns_dict ={
    'timestamp':'time',
    'cnt':'count', 
    't1':'temp_real_C',
    't2':'temp_feels_like_C',
    'hum':'humidity_percent',
    'wind_speed':'wind_speed_kph',
    'weather_code':'weather',
    'is_holiday':'is_holiday',
    'is_weekend':'is_weekend',
    'season':'season'
}

# Renaming the columns to the specified column names
bike_rides.rename(new_columns_dict, axis=1, inplace=True)

In [13]:
# changing the humidity values to percentage (i.e. a value between 0 and 1)
bike_rides.humidity_percent = bike_rides.humidity_percent / 100

In [14]:
# Create a dictionary that maps season indices (0–3) to their names
season_dict = {
    '0.0':'spring',
    '1.0':'summer',
    '2.0':'autumn',
    '3.0':'winter'
}

# Create a dictionary that maps weather codes (integers) to their text labels
weather_dict = {
    '1.0':'Clear',
    '2.0':'Scattered clouds',
    '3.0':'Broken clouds',
    '4.0':'Cloudy',
    '7.0':'Rain',
    '10.0':'Rain with thunderstorm',
    '26.0':'Snowfall'
}

# Changing the seasons column data type to string
bike_rides.season = bike_rides.season.astype('str')
# Mapping the values 0-3 to the actual written seasons
bike_rides.season = bike_rides.season.map(season_dict)

# Changing the weather column data type to string
bike_rides.weather = bike_rides.weather.astype('str')
# Mapping the values to the actual written weathers
bike_rides.weather = bike_rides.weather.map(weather_dict)

In [15]:
# Checking the dataframe to see the changing have worked
bike_rides.head(5)

,time,count,temp_real_C,temp_feels_like_C,humidity_percent,wind_speed_kph,weather,is_holiday,is_weekend,season
0,2015-01-04 00:00:00,182,3.0,2.0,0.930,6.0,Broken clouds,0.0,1.0,winter
1,2015-01-04 01:00:00,138,3.0,2.5,0.930,5.0,Clear,0.0,1.0,winter
2,2015-01-04 02:00:00,134,2.5,2.5,0.965,0.0,Clear,0.0,1.0,winter
3,2015-01-04 03:00:00,72,2.0,2.0,1.000,0.0,Clear,0.0,1.0,winter
4,2015-01-04 04:00:00,47,2.0,0.0,0.930,6.5,Clear,0.0,1.0,winter


In [16]:
# Write the final DataFrame to an Excel file for use in Tableau visualizations
# Save the final DataFrame to 'london_bike_rides_final.xlsx' in the 'Data' sheet for Tableau
bike_rides.to_excel('london_bike_rides_final.xlsx', sheet_name='Data')